In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')  

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
matplotlib.style.use('fivethirtyeight')
matplotlib.style.use('seaborn-talk')


This ipython notebook will illustrate the matching methodology we will use to match patients in the SEER-Medicare linked database. The comparison between patient outcomes between those patients who receive proton beam therapy vs traditional radiation therapy requires as a first step a matching of the two different groups. One method to perform this matching is to use Propensity Score Matching. Here we are going to illustrate a more direct method of matching using KDTrees, a data structure that allows for efficient finding of nearest neighbors in high-dimenional space.

We will first identify all patients that have been diagonised with some type of head and neck cancer as defined by the following primary site codes: Nasopharunx: (C110, C111, C112, C113, C118, C119) Oropharynx: (C091, C098, C099, C100, C101, C102, C103, C104, C108, C109) Salivary gland: (C079-C081, C088-C089) Pituitary gland: (C751) We will illustrate the methodology by using it on the latest release of the SEER data. We will identify all the patients with the above primary site codes. Then we will split the above patient population into two groups as follows: The treatment group, defined as the patients receiving any form of radiation the control group, defined as the patients who did not receive any form of radiation Operationally, these groups are defined by looking at the column RADIATN, which has values:

| Code	| Description |
|:-----------:|:-----------:|
| 0	 | None; diagnosed at autopsy |
| 1|	Beam radiation |
| 2	| Radioacative implants |
| 3	| Radioisotopes |
| 4	| Combination of 1 with 2 or 3 |
| 5	| Radiation, NOS - method or source not specified |
| 6	| Other radiation (1973-1987 cases only) |
| 7 |	Patient or patient's guardian refused radiation therapy |
| 8|	Radiation recommened, unknown if administered |
| 9 |	Unknown if radiation administered |

So that a value of 0 defines the control group, and the values (1,2,3,4,5,6) define the treatment group. We will then perform matching between the resulting treatment and control groups.The difference between treatment patients predicted probability and the control patient's predicted probablity will then be used as the (signed) measurement for performing paired t-tests, so that any observed differences can be classed as statistically significant (or not).

In [3]:
import seerhelp
otherfiles = seerhelp.get_otherfiles()
otherfiles

dfother0 = seerhelp.make_clean_dataframe(otherfiles[0])
dfother1 = seerhelp.make_clean_dataframe(otherfiles[1])
dfother2 = seerhelp.make_clean_dataframe(otherfiles[2])
dfother3 = seerhelp.make_clean_dataframe(otherfiles[3])

dfother = pd.concat([dfother0, dfother1, dfother2,
                     dfother3], ignore_index=True)

#dfother = pd.concat([dfother0, dfother1,dfother3], ignore_index=True)

print(dfother.shape)

del dfother0
del dfother1
del dfother2
del dfother3

(1581838, 141)


In [4]:
pharynxfilter = dfother.PRIMSITE.str.contains('pharynx',case=False,na=False)
pituitaryfilter = dfother.PRIMSITE.str.contains('pituitary',na=False,case=False)
salivaryfilter = dfother.PRIMSITE.str.contains('salivary',na=False,case=False)

dfheadandneck = dfother[pharynxfilter | pituitaryfilter | salivaryfilter].copy()

mask = (dfheadandneck['CSTUMSIZ'] != "Unknown; size not stated; not stated in patient record") & \
(dfheadandneck['CSTUMSIZ'] != "Microscopic focus or foci only; no size of focus is given") & \
(dfheadandneck['CSTUMSIZ'] != "989 millimeters or larger") & \
(dfheadandneck['CSTUMSIZ'] != "Not applicable") & \
(dfheadandneck['AGE_DX'] != "Unknown age") & \
(dfheadandneck['srv_time_mon_flag'] == "Complete dates are available and there are more than 0 days of survival") & \
(dfheadandneck['YEAR_DX'] >= 2004) & \
(dfheadandneck['CSTUMSIZ'] != "Described as less than 1 cm") & \
(dfheadandneck['CSTUMSIZ'] != "Described as less than 2 cm") & \
(dfheadandneck['CSTUMSIZ'] != "Described as less than 3 cm") & \
(dfheadandneck['CSTUMSIZ'] != 'Indicates no msas or no tumor found; for example, when a tumor of a stated primary site is not found, but the tumor has metastasized') & \
(dfheadandneck['CSTUMSIZ'] != "Described as less than 4 cm") & \
(dfheadandneck['CSTUMSIZ'] != "Described as less than 5 cm") & \
(dfheadandneck['CSTUMSIZ'] != "Not applicable") & \
(dfheadandneck['YR_BRTH'] != 'Unknown year of birth') & \
(dfheadandneck['CSTUMSIZ'] != 'Not applicable') & \
(dfheadandneck['CSTUMSIZ'] != 996) & \
(dfheadandneck['CSTUMSIZ'] != 997) & \
(dfheadandneck['CSTUMSIZ'] != 998) & \
(dfheadandneck['REC_NO'] == 1)

dfheadandneck = dfheadandneck[mask]

non_rad = dfheadandneck.RADIATN.str.contains('None',case=False,na=False)
refused_rad = dfheadandneck.RADIATN.str.contains('refused',case=False,na=False)
unknown_rad = dfheadandneck.RADIATN.str.contains('Unknown',case=False,na=False)

In [5]:
dfcontrol = dfheadandneck[non_rad | refused_rad]
dftreatment = dfheadandneck[~(non_rad) & ~(refused_rad) & ~(unknown_rad)]

salivaryfiltercontrol = dfcontrol.PRIMSITE.str.contains('salivary',na=False,case=False)

salivaryfiltertreatment = dftreatment.PRIMSITE.str.contains('salivary',na=False,case=False)

dfsalicontrol = dfcontrol[salivaryfiltercontrol].copy()
dfsalitreatment = dftreatment[salivaryfiltertreatment].copy()

## <font color='steelblue'>Let's look at just the salivary cases</font>



In [6]:
print(dfsalicontrol.shape, dfsalitreatment.shape)

(120, 141) (111, 141)


## <font color='steelblue'>So we are going to match patients in the control group with their nearest neighbors in the treatment group</font>

In order to find the nearest neighbors in the high-dimensional feature space, we need to first do some preprocessing of the data. The first step is to "one-hot-encode" the categorical features.

In [7]:
dfsalicontrol.set_index('PUBCSNUM', inplace=True)
dfsalitreatment.set_index('PUBCSNUM', inplace=True)

controlindices = dfsalicontrol.index
treatmentindices = dfsalitreatment.index

catcols = ['SEX','MAR_STAT','RACEIV','NHIADE','GRADE','PRIMSITE',
          'LATERAL','HST_STGA','HISTREC','MDXRECMP','STAT_REC']

goodcols = ['SEX','MAR_STAT','RACEIV','NHIADE','GRADE','PRIMSITE',
          'LATERAL','HST_STGA','HISTREC','MDXRECMP','STAT_REC',
           'YR_BRTH','AGE_DX','YEAR_DX','CSTUMSIZ',
           'lat','lng']


dfpop = pd.concat([dfsalicontrol, dfsalitreatment],verify_integrity=True)

## <font color='steelblue'>Before matching, let's look at the Kaplan-Meier curves</font>

In [8]:
resgoodcontrol = pd.concat([pd.get_dummies(dfsalicontrol[col],prefix=col) for col in catcols], axis=1)

resgoodtreatment = pd.concat([pd.get_dummies(dfsalitreatment[col],prefix=col) for col in catcols], axis=1)

resgoodpop = pd.concat([pd.get_dummies(dfpop[col],prefix=col) for col in catcols], axis=1)

resgoodcontrol['YR_BRTH'] = dfsalicontrol['YR_BRTH']
resgoodcontrol['AGE_DX'] = dfsalicontrol['AGE_DX']#resgood['sequence_number_central'] = dfsmall['sequence_number_central']
resgoodcontrol['YEAR_DX'] = dfsalicontrol['YEAR_DX']
resgoodcontrol['CSTUMSIZ'] = dfsalicontrol['CSTUMSIZ']
resgoodcontrol['lat'] = dfsalicontrol['lat']
resgoodcontrol['lng'] = dfsalicontrol['lng']
resgoodcontrol['srv_time_mon'] = dfsalicontrol['srv_time_mon']


resgoodtreatment['YR_BRTH'] = dfsalitreatment['YR_BRTH']
resgoodtreatment['AGE_DX'] = dfsalitreatment['AGE_DX']#resgood['sequence_number_central'] = dfsmall['sequence_number_central']
resgoodtreatment['YEAR_DX'] = dfsalitreatment['YEAR_DX']
resgoodtreatment['CSTUMSIZ'] = dfsalitreatment['CSTUMSIZ']
resgoodtreatment['lat'] = dfsalitreatment['lat']
resgoodtreatment['lng'] = dfsalitreatment['lng']
resgoodtreatment['srv_time_mon'] = dfsalitreatment['srv_time_mon']


resgoodpop['YR_BRTH'] = dfpop['YR_BRTH']
resgoodpop['AGE_DX'] = dfpop['AGE_DX']#resgood['sequence_number_central'] = dfsmall['sequence_number_central']
resgoodpop['YEAR_DX'] = dfpop['YEAR_DX']
resgoodpop['CSTUMSIZ'] = dfpop['CSTUMSIZ']
resgoodpop['lat'] = dfpop['lat']
resgoodpop['lng'] = dfpop['lng']
resgoodpop['srv_time_mon'] = dfpop['srv_time_mon']